In [1]:
import itertools
import re
import math

import numpy as np
import networkx as nx
from io import StringIO

from functools import lru_cache
from collections import Counter, defaultdict
from lib import level_a, level_b, level_ab

In [2]:
@level_ab(1, apply=int, test=("199\n200\n208\n210\n200\n207\n240\n269\n260\n263", 7, 5))
def solve(num, level):
    if level:
        num = num[2:] + num[1:-1] + num[:-2]  # 3-frame-windowing
    return ( ( num[1:] - num[:-1] ) > 0).sum()

Part a already solved with same answer: 1184
Part b already solved with same answer: 1158


In [3]:
@level_ab(2, test=('forward 5\ndown 5\nforward 8\nup 3\ndown 8\nforward 2', 150, 900))
def solve(lines, level):
    DIRS = {"forward": 1j, "up": 1, "down": -1}
    pos = aim = 0
    for line in lines:
        dir, dist = line.split()
        dir, dist = DIRS[dir], int(dist)
        if level:
            if dir.real: 
                aim += dist * dir
            else:
                pos += dist + dist * aim * dir
        else:
            pos += dir * dist
    return np.absolute([pos.real, pos.imag]).prod(dtype=int)

Part a already solved with same answer: 1694130
Part b already solved with same answer: 1698850445


In [4]:
def inner(arr, level):
    for pos in range(arr.shape[1]):
        curr = arr[:,pos].mean(axis=0) >= 0.5  # unlike np.round, round UP on 0.5
        curr = int(curr == level)  # invert for epsilon
        arr = arr[arr[:,pos] == curr]  # filter array at position pos
        if len(arr) == 1:
            return arr[0]

@level_ab(3, apply=lambda line: [int(i) for i in line], test=('00100\n11110\n10110\n10111\n10101\n01111\n00111\n11100\n10000\n11001\n00010\n01010', 198, 230))
def solve(num, level):
    bin2dec = lambda x: int("".join(map(str, x)), 2)

    if level:
        return bin2dec(inner(num, 1)) * bin2dec(inner(num, 0))
    else:
        gamma = num.mean(axis=0).round().astype(int)
        epsilon = 1 - gamma  # boolean invert
        return bin2dec(gamma) * bin2dec(epsilon)

Part a already solved with same answer: 2743844
Part b already solved with same answer: 6677951


In [5]:
@level_ab(4, sep="\n\n", test=('7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1\n\n22 13 17 11  0\n 8  2 23  4 24\n21  9 14 16  7\n 6 10  3 18  5\n 1 12 20 15 19\n\n 3 15  0  2 22\n 9 18 13 17  5\n19  8  7 25 23\n20 11 10 24  4\n14 21 16 12  6\n\n14 21 17 24  4\n10 16 15  9 19\n18  8 23 26 20\n22 11 13  6  5\n 2  0 12  3  7', 4512, 1924))
def solve(chunks, level):
    numbers, *fields = chunks
    fields = [np.genfromtxt(field.splitlines(), dtype=int) for field in fields]
    winners = set()
    for number in map(int, numbers.split(",")):
        for i in range(len(fields)):
            field = fields[i]
            fields[i][field==number] = -1
            mask = fields[i] == -1
            if (mask.sum(axis=0) == 5).any() or (mask.sum(axis=1) == 5).any():
                winners.add(i)
                if not level or len(winners) == len(fields):
                    return number * field[~mask].sum()

Part a already solved with same answer: 60368
Part b already solved with same answer: 17435


In [6]:
@level_ab(5, test=('0,9 -> 5,9\n8,0 -> 0,8\n9,4 -> 3,4\n2,2 -> 2,1\n7,0 -> 7,4\n6,4 -> 2,0\n0,9 -> 2,9\n3,4 -> 1,4\n0,0 -> 8,8\n5,5 -> 8,2', 5, 12))
def solve(lines, level):
    field = defaultdict(int)
    for line in lines:
        start, end = [complex(*[int(i) for i in part.split(",")]) for part in line.split(" -> ")]
        dist = end - start
        dlen = int(max(abs(dist.real), abs(dist.imag)))
        if not dist.real or not dist.imag or (level and dist.real / dist.imag % 1 == 0):
            for d in range(dlen+1):
                field[start + (dist/dlen) * d] += 1                
    return sum([v > 1 for v in field.values()])

Part a already solved with same answer: 5835
Part b already solved with same answer: 17013


In [7]:
@level_ab(6, sep=",", apply=int, test=("3,4,3,1,2", 5934, 26984457539))
def solve(num, level):
    cnt = Counter(num)
    for i in range(256 if level else 80):
        cnt = Counter({k-1: v for k, v in cnt.items()})
        cnt[6] += cnt.get(-1, 0)
        cnt[8] += cnt.get(-1, 0)
        cnt[-1] = 0
    return sum(cnt.values())

Part a already solved with same answer: 350149
Part b already solved with same answer: 1590327954513


In [8]:
# alt solution with matrix kernel power
def solve_np(data, days):
    cnt = np.bincount(data, minlength=9)
    kernel = np.pad(np.identity(8, dtype=np.int64), ((1, 0), (0, 1)))  # discrete projection: t_x -> t_(x+1) = 1 -> 0, 2 -> 1
    kernel[0, [6,8]] = 1  # extra projection: 0 -> 6, 8
    kernel = np.linalg.matrix_power(kernel.astype(object), days)  # astype object to use python builtin unlimited size ints, works fine with a million
    return (cnt @ kernel).sum()

# len(str(solve_np((3,4,3,1,2), 2**20)))  # 39674

In [9]:
@level_ab(7, sep=",", apply=int, test=("16,1,2,0,4,2,7,1,2,14", 37, 168))
def solve(num, level):
    distance = lambda n: n*(n+1)//2 if level else n
    return min([distance(np.abs(num-i)).sum() 
                for i in range(min(num), max(num))])

Part a already solved with same answer: 329389
Part b already solved with same answer: 86397080


In [ ]:
@level_ab(8, sep=",", test=("""TODO""", 0, 0))
def solve(data, level):
    return data